In [17]:
import pandas as pd

tlf_dict = pd.read_pickle("./data/tlf_dict")
tlf_tuplelist = list(tlf_dict.items())
tlf_tuplelist_keys = [key[0] for key in tlf_tuplelist]

In [25]:
tlf_tuplelist_keys[20000]

'정목'

In [26]:
train_load_df = pd.read_pickle("./data/train_df.pkl")
train_sent = train_load_df['sentence']
train_label = train_load_df['label']

In [28]:
N = 12
lab = [0 for _ in range(N)]
for label in train_label:
    lab[label] += 1
print(lab)

[368737, 161471, 82812, 67875, 64002, 61957, 24679, 20823, 5668, 4945, 4200, 13356]


In [32]:
minimum = min(lab)
# [368737, 161471, 82812, 67875, 64002, 61957, 24679, 20823, 5668, 4945, 4200, 13356]
minimum = 20000

In [33]:
import numpy as np

curr = [0 for _ in range(N)]
x_train = []
y_train = []
for x, y in zip(train_sent, train_label):
    if curr[y] > minimum:
        continue
    else:
        curr[y] += 1
        x_train.append(x)
        y_train.append(y)

x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train.shape)
print(y_train.shape)
        

(188177,)
(188177,)


/tmp/ipykernel_741440/1029815814.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array(x_train)


In [35]:
import pickle

with open('./data/x_train_undersamp', 'wb') as file_pi:
    pickle.dump(x_train, file_pi)
with open('./data/y_train_undersamp', 'wb') as file_pi:
    pickle.dump(y_train, file_pi)

In [49]:
from tensorflow.keras.layers import Dense, Dropout, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model

vocab_size = 20000
txtInput = Input(shape=(vocab_size, N, 1, ), name = 'txt_input')
img = Conv2D(64, (3,3), padding='same', name='block1_conv1', activation='relu')(txtInput)
img = Conv2D(64, (3,3), padding='same', name='block1_conv2', activation='relu')(img)
#img = Conv2D(64, (3,3), padding='same', name='block1_conv3', activation='relu')(img)
img = MaxPooling2D((4,1), name='block1_pool')(img)
img = Conv2D(128, (3,3), padding='same', name='block2_conv1', activation='relu')(img)
img = Conv2D(128, (3,3), padding='same', name='block2_conv2', activation='relu')(img)
#img = Conv2D(128, (3,3), padding='same', name='block2_conv3', activation='relu')(img)
img = MaxPooling2D((4,1), name='block2_pool')(img)
img = Conv2D(256, (3,3), padding='same', name='block3_conv1', activation='relu')(img)
img = Conv2D(256, (3,3), padding='same', name='block3_conv2', activation='relu')(img)
#img = Conv2D(256, (3,3), padding='same', name='block3_conv3', activation='relu')(img)
img = MaxPooling2D((4,1), name='block3_pool')(img)
img = Conv2D(512, (3,3), padding='same', name='block4_conv1', activation='relu')(img)
img = Conv2D(512, (3,3), padding='same', name='block4_conv2', activation='relu')(img)
img = MaxPooling2D((4,1), name='block4_pool')(img)
img = Conv2D(1024, (3,3), padding='same', name='block5_conv1', activation='relu')(img)
img = Conv2D(1024, (3,3), padding='same', name='block5_conv2', activation='relu')(img)
img = MaxPooling2D((4,1), name='block5_pool')(img)
img = Flatten()(img)
output = Dense(N, activation='softmax')(img)

model = Model(inputs = txtInput, outputs = output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 txt_input (InputLayer)      [(None, 20000, 12, 1)]    0         
                                                                 
 block1_conv1 (Conv2D)       (None, 20000, 12, 64)     640       
                                                                 
 block1_conv2 (Conv2D)       (None, 20000, 12, 64)     36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 5000, 12, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 5000, 12, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 5000, 12, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 1250, 12, 128)     0  

In [12]:
from tensorflow.keras.layers import concatenate, Dense, Dropout, Input, Conv2D, MaxPooling2D, Flatten, Embedding, GRU
from tensorflow.keras.models import Model

vocab_size = 50000 # 사용할 단어의 개수 (빈도 순)
N = 12 # 분류할 label의 종류
usage = 0.8 # 현재 data의 크기가 커서, generator를 만들기 전까지 이걸로 사용
max_len = 520 # padding을 맞출 문장의 최대 길이
embedding_dim = 100 # 임베딩 차원
dropout_rate = 0.3

txtInput = Input(shape=(max_len,), name='txt_input')
txt = Embedding(vocab_size, embedding_dim, input_length=max_len)(txtInput)
txt = GRU(512, return_sequences=True)(txt)
txt = Dropout(dropout_rate)(txt)
txt = GRU(512, return_sequences=True)(txt)
txt = Dropout(dropout_rate)(txt)
txt = GRU(512)(txt)
txt = Dense(512, activation='relu')(txt)

imgInput = Input(shape=(vocab_size, N, 1, ), name = 'img_input')
img = Conv2D(64, (3,3), padding='same', name='block1_conv1', activation='relu')(imgInput)
img = Conv2D(64, (3,3), padding='same', name='block1_conv2', activation='relu')(img)
#img = Conv2D(64, (3,3), padding='same', name='block1_conv3', activation='relu')(img)
img = MaxPooling2D((4,1), name='block1_pool')(img)
img = Conv2D(128, (3,3), padding='same', name='block2_conv1', activation='relu')(img)
img = Conv2D(128, (3,3), padding='same', name='block2_conv2', activation='relu')(img)
#img = Conv2D(128, (3,3), padding='same', name='block2_conv3', activation='relu')(img)
img = MaxPooling2D((4,1), name='block2_pool')(img)
img = Conv2D(256, (3,3), padding='same', name='block3_conv1', activation='relu')(img)
img = Conv2D(256, (3,3), padding='same', name='block3_conv2', activation='relu')(img)
#img = Conv2D(256, (3,3), padding='same', name='block3_conv3', activation='relu')(img)
img = MaxPooling2D((4,1), name='block3_pool')(img)
img = Conv2D(512, (3,3), padding='same', name='block4_conv1', activation='relu')(img)
img = Conv2D(512, (3,3), padding='same', name='block4_conv2', activation='relu')(img)
img = MaxPooling2D((4,1), name='block4_pool')(img)
img = Conv2D(1024, (3,3), padding='same', name='block5_conv1', activation='relu')(img)
img = Conv2D(1024, (3,3), padding='same', name='block5_conv2', activation='relu')(img)
img = MaxPooling2D((4,1), name='block5_pool')(img)
img = Flatten()(img)

output = concatenate([img, txt], axis=-1)
output = Dropout(dropout_rate)(output)
output = Dense(N, activation='softmax')(output)

model = Model(inputs = [imgInput, txtInput], outputs = output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img_input (InputLayer)         [(None, 50000, 12,   0           []                               
                                1)]                                                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 50000, 12, 6  640         ['img_input[0][0]']              
                                4)                                                                
                                                                                                  
 block1_conv2 (Conv2D)          (None, 50000, 12, 6  36928       ['block1_conv1[0][0]']           
                                4)                                                          